In [84]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

In [85]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [86]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


## Load Dataset

In [87]:
train_data = pd.read_csv('/opt/ml/input/data/train/train.csv')

In [102]:
print(train_data.dtypes)
print(train_data.shape)

train_data.head()

id        object
gender    object
race      object
age        int64
path      object
dtype: object
(2700, 5)


,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59


In [103]:
train_img = "/opt/ml/input/data/train/images"

img_list = os.listdir(train_img)

print(len(img_list))

5400


## EDA

In [90]:
train_data['gender'].value_counts()

female    1658
male      1042
Name: gender, dtype: int64

In [91]:
train_data['race'].value_counts()

Asian    2700
Name: race, dtype: int64

race col은 뺄 것

In [92]:
new_train_data = train_data.drop(['race'], axis = 1)

In [94]:
new_train_data

,id,gender,age,path
0,000001,female,45,000001_female_Asian_45
1,000002,female,52,000002_female_Asian_52
2,000004,male,54,000004_male_Asian_54
3,000005,female,58,000005_female_Asian_58
4,000006,female,59,000006_female_Asian_59
...,...,...,...,...
2695,006954,male,19,006954_male_Asian_19
2696,006955,male,19,006955_male_Asian_19
2697,006956,male,19,006956_male_Asian_19
2698,006957,male,20,006957_male_Asian_20


### 라벨링 기준

1.Mask: [wear, incorrect, Not wear] = [0, 6, 12]
2.Gender: [Male, Female] = [0, 3]
3.Age: [<30, 30 by 60, 60] = [0, 1, 2]

## Data Pre-processing

In [97]:
new_train_data['gender'] = new_train_data['gender'].replace('male', 1).replace('female', 0)

In [99]:
def age(x):
    if x < 30:
        return 0
    elif 30 <= x < 60:
        return 1
    else:
        return 2

In [100]:
new_train_data['age(encoding)'] = new_train_data['age'].apply(lambda x: age(x))

In [101]:
new_train_data.head()

,id,gender,age,path,age(encoding)
0,000001,0,45,000001_female_Asian_45,1
1,000002,0,52,000002_female_Asian_52,1
2,000004,1,54,000004_male_Asian_54,1
3,000005,0,58,000005_female_Asian_58,1
4,000006,0,59,000006_female_Asian_59,1
